In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
harris_file = "harris_timeline.json"

with open(harris_file, 'r') as file:
    harris_data = json.load(file)

In [3]:
harris_data[0]

{'created_at': 'Thu Oct 22 00:41:00 +0000 2020',
 'id': 1319076251483734018,
 'id_str': '1319076251483734018',
 'text': "We're just days away from the end of the election. Help us bring it home by donating $20 today:… https://t.co/OAzXOecJyB",
 'truncated': True,
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [],
  'urls': [{'url': 'https://t.co/OAzXOecJyB',
    'expanded_url': 'https://twitter.com/i/web/status/1319076251483734018',
    'display_url': 'twitter.com/i/web/status/1…',
    'indices': [97, 120]}]},
 'source': '<a href="https://studio.twitter.com" rel="nofollow">Twitter Media Studio</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 30354991,
  'id_str': '30354991',
  'name': 'Kamala Harris',
  'screen_name': 'KamalaHarris',
  'location': 'California',
  'description': 'U.S. Senator and Democratic candidate for Vice Presi

In [4]:
df_harris = pd.DataFrame([t['created_at'] for t in harris_data], columns=['create_time'])
df_harris['text'] = [t['text'] for t in harris_data]
df_harris['retweets'] = [t['retweet_count'] for t in harris_data]
df_harris['favorites'] = [t['favorite_count'] for t in harris_data]

In [5]:
df_harris.head()

,create_time,text,retweets,favorites
0,Thu Oct 22 00:41:00 +0000 2020,We're just days away from the end of the elect...,733,4221
1,Wed Oct 21 23:29:52 +0000 2020,545 children. \n\nThis is outrageous and a sta...,3100,17262
2,Wed Oct 21 22:51:39 +0000 2020,This is our moment to do something for our fam...,982,5514
3,Wed Oct 21 21:47:21 +0000 2020,.@BarackObama knows that the election is happe...,1621,9163
4,Wed Oct 21 20:21:01 +0000 2020,RT @JoeBiden: Tune in as @BarackObama sits dow...,4097,0


In [8]:
df_harris.tail()

,create_time,text,retweets,favorites
195,Fri Oct 02 20:04:59 +0000 2020,"RT @JoeBiden: Tonight, @BarackObama, @KamalaHa...",5771,0
196,Fri Oct 02 17:47:11 +0000 2020,Both @DouglasEmhoff and I were tested for COVI...,18351,181235
197,Fri Oct 02 16:31:50 +0000 2020,RT @JoeBiden: I’m happy to report that Jill an...,135093,0
198,Fri Oct 02 13:17:04 +0000 2020,Doug and I join Joe Biden and Dr. Biden in wis...,13727,176398
199,Fri Oct 02 02:23:15 +0000 2020,Could not be more thrilled that one of my favo...,2175,14039
